<span style="font-weight:bold; font-size: 3rem; color:#0047AB;">Feature pipeline updating hourly</span>

## 🗒️ This notebook is divided into the following sections:

1. Retrieves real-time weather and electricity data and inserts the new data into respective feature groups


## <span style='color:#ff5f27'> Imports</span>

In [1]:
import pandas as pd
import os
import numpy as np
import requests
from datetime import datetime, timedelta
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime, timedelta
import time
import json
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.ticker import MultipleLocator
import openmeteo_requests
import requests_cache
from retry_requests import retry
import hsfs
from pathlib import Path
from dotenv import load_dotenv
import hopsworks
import sys

root_dir = Path().resolve().parent
sys.path.append(str(root_dir))

from format_data import format_weather_data, format_price_data, process_weather_data
from get_electricity_prices import get_data
from get_weather_data import get_historical_weather, get_weather_forecast
from entsoe_data import fetch_historical_data, ensure_valid_series
from util import modify_weather_df, modify_entsoe_df, create_lagging_columns, add_future_price_column




### Connecting to Hopsworks and Entose API

In [2]:
if not os.getenv("CI"): 
    load_dotenv()
    
hopsworks_api = os.getenv("HOPSWORKS_API_KEY")
entose_api = os.getenv("ENTSOE_API")

if not hopsworks_api:
    raise ValueError("HOPSWORKS_API_KEY is not set.")
if not entose_api:
    raise ValueError("ENTSOE_API is not set.")

os.environ["ENTSOE_API"] = entose_api
os.environ["HOPSWORKS_API_KEY"] = hopsworks_api

project = hopsworks.login()
fs = project.get_feature_store() 
print(f"Connected to project: {project.name}")


2025-01-09 11:43:22,409 INFO: Initializing external client
2025-01-09 11:43:22,409 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-09 11:43:24,253 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1164446
Connected to project: oskaralf


In [3]:
start_date = datetime.now().strftime('%Y-%m-%d')
end_date = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')

## <span style='color:#ff5f27'> Get latest weather and electricity data</span>

### Get weather data and format it

In [4]:
forecast = get_weather_forecast("Stockholm", "2022-11-01", "2025-01-03", 59.3294, 18.0687)

formatted_forecast_df = process_weather_data(forecast)
formatted_forecast_df = modify_weather_df(formatted_forecast_df)
formatted_forecast_df = create_lagging_columns(formatted_forecast_df)
print(formatted_forecast_df.dtypes)

Coordinates 59.32889938354492°N 18.072357177734375°E
Elevation 24.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
number of rows before dropping na:  168
number of rows after dropping na:  167
date                   datetime64[ns]
temperature_2m                float32
precipitation                 float32
wind_speed_100m               float32
temperature_2m_lag            float32
precipitation_lag             float32
wind_speed_100m_lag           float32
dtype: object


### Retreive feature groups

In [5]:

weather_fg = fs.get_feature_group(
    name='weather_data_3',
    version=1,
)
entsoe_fg = fs.get_feature_group(
    name='entsoe_data_3',
    version=1,
)

In [6]:
print(formatted_forecast_df.isna().sum())

date                   0
temperature_2m         0
precipitation          0
wind_speed_100m        0
temperature_2m_lag     0
precipitation_lag      0
wind_speed_100m_lag    0
dtype: int64


In [7]:
formatted_forecast_df = formatted_forecast_df.dropna()
weather_fg.insert(formatted_forecast_df, write_options={"wait_for_job": True})

2025-01-09 11:43:27,109 INFO: 	6 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1164446/fs/1155149/fg/1393654


Uploading Dataframe: 100.00% |██████████| Rows 167/167 | Elapsed Time: 00:01 | Remaining Time: 00:00


2025-01-09 11:43:35,528 WARNING: UserWarning: Materialization job is already running, aborting new execution.Please wait for the current execution to finish before triggering a new one.You can check the status of the current execution using `fg.materialization_job.get_state()`.or `fg.materialization_job.get_final_state()` or check it out in the Hopsworks UI.at https://c.app.hopsworks.ai:443/p/1164446/jobs/named/weather_data_3_1_offline_fg_materialization.
Use fg.materialization_job.run(args=-op offline_fg_materialization -path hdfs:///Projects/oskaralf/Resources/jobs/weather_data_3_1_offline_fg_materialization/config_1736196513098) to trigger the materialization job again.



(Job('weather_data_3_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "temperature_2m_lag",
           "min_value": -100.0,
           "max_value": 500.0
         },
         "meta": {
           "expectationId": 696123
         }
       },
       "result": {
         "observed_value": -8.649999618530273,
         "element_count": 167,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-01-09T10:43:27.000103Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_t

### Fetch latest electricity data from Entsoe, and insert it in feature group

In [8]:
entsoe_df = fetch_historical_data(entose_api, start_date, end_date)

[DEBUG] Successfully fetched load data for finland.
[DEBUG] Successfully fetched load data for norway.
[DEBUG] Successfully fetched load data for denmark.
[DEBUG] Successfully fetched cross-border flows SE3 to finland.
[DEBUG] Successfully fetched cross-border flows finland to SE3.
[DEBUG] Successfully fetched cross-border flows SE3 to norway.
[DEBUG] Successfully fetched cross-border flows norway to SE3.
[DEBUG] Successfully fetched cross-border flows SE3 to denmark.
[DEBUG] Successfully fetched cross-border flows denmark to SE3.


In [9]:
entsoe_df

,load_se3,load_finland,load_norway,load_denmark,prices,total_generation_Biomass,total_generation_Fossil Gas,total_generation_Fossil Hard coal,total_generation_Fossil Oil,total_generation_Hydro Run-of-river and poundage,...,total_generation_Waste,total_generation_Wind Offshore,total_generation_Wind Onshore,flows_se3_to_finland,flows_finland_to_se3,flows_se3_to_norway,flows_norway_to_se3,flows_se3_to_denmark,flows_denmark_to_se3,date
index,,,,,,,,,,,,,,,,,,,,,
2025-01-08 23:00:00+00:00,11066.0,10739.00,5568.0,2821.0,101.44,306.0,257.0,118.0,12.0,4.0,...,67.0,675.0,750.0,568.0,0.0,0.0,106.0,714.0,0.0,2025-01-09 00:00:00
2025-01-09 00:00:00+00:00,10873.0,10616.50,5457.0,2762.0,100.00,305.0,265.0,117.0,11.0,4.0,...,66.0,645.0,660.0,647.0,0.0,0.0,169.0,714.0,0.0,2025-01-09 01:00:00
2025-01-09 01:00:00+00:00,10789.0,10608.50,5368.0,2707.0,98.77,318.0,266.0,117.0,11.0,4.0,...,68.0,576.0,567.0,665.0,0.0,0.0,295.0,714.0,0.0,2025-01-09 02:00:00
2025-01-09 02:00:00+00:00,10696.0,10551.75,5256.0,2613.0,95.22,302.0,268.0,116.0,11.0,4.0,...,70.0,497.0,426.0,696.0,0.0,0.0,334.0,714.0,0.0,2025-01-09 03:00:00
2025-01-09 03:00:00+00:00,10751.0,10791.00,5224.0,2543.0,98.28,303.0,271.0,121.0,11.0,4.0,...,69.0,474.0,326.0,572.0,0.0,0.0,659.0,714.0,0.0,2025-01-09 04:00:00
2025-01-09 04:00:00+00:00,11195.0,11416.75,5392.0,2571.0,102.65,302.0,272.0,162.0,11.0,4.0,...,69.0,276.0,256.0,620.0,0.0,0.0,1300.0,714.0,0.0,2025-01-09 05:00:00
2025-01-09 05:00:00+00:00,12247.0,11682.25,5841.0,2896.0,133.54,301.0,328.0,262.0,17.0,4.0,...,78.0,324.0,196.0,504.0,0.0,0.0,1796.0,715.0,0.0,2025-01-09 06:00:00
2025-01-09 06:00:00+00:00,13130.0,11698.00,6144.0,3227.0,148.80,301.0,394.0,326.0,21.0,4.0,...,96.0,176.0,175.0,39.0,148.0,0.0,1509.0,715.0,0.0,2025-01-09 07:00:00
2025-01-09 07:00:00+00:00,13462.0,11714.50,6193.0,3357.0,164.88,315.0,394.0,308.0,19.0,4.0,...,96.0,106.0,137.0,0.0,501.0,0.0,1395.0,715.0,0.0,2025-01-09 08:00:00


In [10]:
print(entsoe_df.dtypes)

load_se3                                            float64
load_finland                                        float64
load_norway                                         float64
load_denmark                                        float64
prices                                              float64
total_generation_Biomass                            float64
total_generation_Fossil Gas                         float64
total_generation_Fossil Hard coal                   float64
total_generation_Fossil Oil                         float64
total_generation_Hydro Run-of-river and poundage    float64
total_generation_Other renewable                    float64
total_generation_Solar                              float64
total_generation_Waste                              float64
total_generation_Wind Offshore                      float64
total_generation_Wind Onshore                       float64
flows_se3_to_finland                                float64
flows_finland_to_se3                    

In [11]:

entsoe_df.columns = entsoe_df.columns.str.lower()
entsoe_df.columns = entsoe_df.columns.str.lower().str.replace(' ', '_').str.replace('-', '_')
entsoe_df['date'] = pd.to_datetime(entsoe_df['date'])
entsoe_df['date'] = pd.to_datetime(entsoe_df['date']).dt.tz_localize('UTC').dt.tz_convert(None)
#entsoe_df = entsoe_df.dropna()
print(entsoe_df.dtypes)
print(entsoe_df.tail(5))

load_se3                                                   float64
load_finland                                               float64
load_norway                                                float64
load_denmark                                               float64
prices                                                     float64
total_generation_biomass                                   float64
total_generation_fossil_gas                                float64
total_generation_fossil_hard_coal                          float64
total_generation_fossil_oil                                float64
total_generation_hydro_run_of_river_and_poundage           float64
total_generation_other_renewable                           float64
total_generation_solar                                     float64
total_generation_waste                                     float64
total_generation_wind_offshore                             float64
total_generation_wind_onshore                              flo

In [12]:
entsoe_df

,load_se3,load_finland,load_norway,load_denmark,prices,total_generation_biomass,total_generation_fossil_gas,total_generation_fossil_hard_coal,total_generation_fossil_oil,total_generation_hydro_run_of_river_and_poundage,...,total_generation_waste,total_generation_wind_offshore,total_generation_wind_onshore,flows_se3_to_finland,flows_finland_to_se3,flows_se3_to_norway,flows_norway_to_se3,flows_se3_to_denmark,flows_denmark_to_se3,date
index,,,,,,,,,,,,,,,,,,,,,
2025-01-08 23:00:00+00:00,11066.0,10739.00,5568.0,2821.0,101.44,306.0,257.0,118.0,12.0,4.0,...,67.0,675.0,750.0,568.0,0.0,0.0,106.0,714.0,0.0,2025-01-09 00:00:00
2025-01-09 00:00:00+00:00,10873.0,10616.50,5457.0,2762.0,100.00,305.0,265.0,117.0,11.0,4.0,...,66.0,645.0,660.0,647.0,0.0,0.0,169.0,714.0,0.0,2025-01-09 01:00:00
2025-01-09 01:00:00+00:00,10789.0,10608.50,5368.0,2707.0,98.77,318.0,266.0,117.0,11.0,4.0,...,68.0,576.0,567.0,665.0,0.0,0.0,295.0,714.0,0.0,2025-01-09 02:00:00
2025-01-09 02:00:00+00:00,10696.0,10551.75,5256.0,2613.0,95.22,302.0,268.0,116.0,11.0,4.0,...,70.0,497.0,426.0,696.0,0.0,0.0,334.0,714.0,0.0,2025-01-09 03:00:00
2025-01-09 03:00:00+00:00,10751.0,10791.00,5224.0,2543.0,98.28,303.0,271.0,121.0,11.0,4.0,...,69.0,474.0,326.0,572.0,0.0,0.0,659.0,714.0,0.0,2025-01-09 04:00:00
2025-01-09 04:00:00+00:00,11195.0,11416.75,5392.0,2571.0,102.65,302.0,272.0,162.0,11.0,4.0,...,69.0,276.0,256.0,620.0,0.0,0.0,1300.0,714.0,0.0,2025-01-09 05:00:00
2025-01-09 05:00:00+00:00,12247.0,11682.25,5841.0,2896.0,133.54,301.0,328.0,262.0,17.0,4.0,...,78.0,324.0,196.0,504.0,0.0,0.0,1796.0,715.0,0.0,2025-01-09 06:00:00
2025-01-09 06:00:00+00:00,13130.0,11698.00,6144.0,3227.0,148.80,301.0,394.0,326.0,21.0,4.0,...,96.0,176.0,175.0,39.0,148.0,0.0,1509.0,715.0,0.0,2025-01-09 07:00:00
2025-01-09 07:00:00+00:00,13462.0,11714.50,6193.0,3357.0,164.88,315.0,394.0,308.0,19.0,4.0,...,96.0,106.0,137.0,0.0,501.0,0.0,1395.0,715.0,0.0,2025-01-09 08:00:00


In [13]:
entsoe_df = modify_entsoe_df(entsoe_df)
entsoe_df = create_lagging_columns(entsoe_df)
entsoe_df = add_future_price_column(entsoe_df)

number of rows before dropping na:  10
number of rows after dropping na:  9


In [14]:
print(entsoe_df.dtypes)
entsoe_df

prices                            float64
date                       datetime64[ns]
imported_energy                   float64
total_load                        float64
total_generation_se               float64
prices_lag                        float64
imported_energy_lag               float64
total_load_lag                    float64
total_generation_se_lag           float64
future_price                      float64
dtype: object


,prices,date,imported_energy,total_load,total_generation_se,prices_lag,imported_energy_lag,total_load_lag,total_generation_se_lag,future_price
index,,,,,,,,,,
2025-01-09 00:00:00+00:00,100.00,2025-01-09 01:00:00,-1192.0,29708.50,2074.0,101.44,-1176.0,30194.00,2190.0,98.77
2025-01-09 01:00:00+00:00,98.77,2025-01-09 02:00:00,-1084.0,29472.50,1928.0,100.00,-1192.0,29708.50,2074.0,95.22
2025-01-09 02:00:00+00:00,95.22,2025-01-09 03:00:00,-1076.0,29116.75,1695.0,98.77,-1084.0,29472.50,1928.0,98.28
2025-01-09 03:00:00+00:00,98.28,2025-01-09 04:00:00,-627.0,29309.00,1580.0,95.22,-1076.0,29116.75,1695.0,102.65
2025-01-09 04:00:00+00:00,102.65,2025-01-09 05:00:00,-34.0,30574.75,1353.0,98.28,-627.0,29309.00,1580.0,133.54
2025-01-09 05:00:00+00:00,133.54,2025-01-09 06:00:00,577.0,32666.25,1513.0,102.65,-34.0,30574.75,1353.0,148.80
2025-01-09 06:00:00+00:00,148.80,2025-01-09 07:00:00,903.0,34199.00,1496.0,133.54,577.0,32666.25,1513.0,164.88
2025-01-09 07:00:00+00:00,164.88,2025-01-09 08:00:00,1181.0,34726.50,1383.0,148.80,903.0,34199.00,1496.0,156.15
2025-01-09 08:00:00+00:00,156.15,2025-01-09 09:00:00,733.0,34540.00,1503.0,164.88,1181.0,34726.50,1383.0,NaN


In [15]:
entsoe_fg.insert(entsoe_df, write_options={"wait_for_job": True})

2025-01-09 11:43:49,576 INFO: 	9 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1164446/fs/1155149/fg/1393655


Uploading Dataframe: 100.00% |██████████| Rows 9/9 | Elapsed Time: 00:01 | Remaining Time: 00:00


2025-01-09 11:43:56,654 WARNING: UserWarning: Materialization job is already running, aborting new execution.Please wait for the current execution to finish before triggering a new one.You can check the status of the current execution using `fg.materialization_job.get_state()`.or `fg.materialization_job.get_final_state()` or check it out in the Hopsworks UI.at https://c.app.hopsworks.ai:443/p/1164446/jobs/named/entsoe_data_3_1_offline_fg_materialization.
Use fg.materialization_job.run(args=-op offline_fg_materialization -path hdfs:///Projects/oskaralf/Resources/jobs/entsoe_data_3_1_offline_fg_materialization/config_1736196531372) to trigger the materialization job again.



(Job('entsoe_data_3_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "prices_lag",
           "min_value": -1000,
           "max_value": 1000,
           "strict_min": true
         },
         "meta": {
           "expectationId": 696127
         }
       },
       "result": {
         "observed_value": 95.22,
         "element_count": 9,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-01-09T10:43:49.000575Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_

In [16]:
print(entsoe_df.dtypes)
entsoe_df

prices                            float64
date                       datetime64[ns]
imported_energy                   float64
total_load                        float64
total_generation_se               float64
prices_lag                        float64
imported_energy_lag               float64
total_load_lag                    float64
total_generation_se_lag           float64
future_price                      float64
dtype: object


,prices,date,imported_energy,total_load,total_generation_se,prices_lag,imported_energy_lag,total_load_lag,total_generation_se_lag,future_price
index,,,,,,,,,,
2025-01-09 00:00:00+00:00,100.00,2025-01-09 01:00:00,-1192.0,29708.50,2074.0,101.44,-1176.0,30194.00,2190.0,98.77
2025-01-09 01:00:00+00:00,98.77,2025-01-09 02:00:00,-1084.0,29472.50,1928.0,100.00,-1192.0,29708.50,2074.0,95.22
2025-01-09 02:00:00+00:00,95.22,2025-01-09 03:00:00,-1076.0,29116.75,1695.0,98.77,-1084.0,29472.50,1928.0,98.28
2025-01-09 03:00:00+00:00,98.28,2025-01-09 04:00:00,-627.0,29309.00,1580.0,95.22,-1076.0,29116.75,1695.0,102.65
2025-01-09 04:00:00+00:00,102.65,2025-01-09 05:00:00,-34.0,30574.75,1353.0,98.28,-627.0,29309.00,1580.0,133.54
2025-01-09 05:00:00+00:00,133.54,2025-01-09 06:00:00,577.0,32666.25,1513.0,102.65,-34.0,30574.75,1353.0,148.80
2025-01-09 06:00:00+00:00,148.80,2025-01-09 07:00:00,903.0,34199.00,1496.0,133.54,577.0,32666.25,1513.0,164.88
2025-01-09 07:00:00+00:00,164.88,2025-01-09 08:00:00,1181.0,34726.50,1383.0,148.80,903.0,34199.00,1496.0,156.15
2025-01-09 08:00:00+00:00,156.15,2025-01-09 09:00:00,733.0,34540.00,1503.0,164.88,1181.0,34726.50,1383.0,NaN
